In [ ]:
from kg_manager_optimized import run_analysis, DEFAULT_TTL_PATH
import os
import sys

# Pfad zum übergeordneten Verzeichnis ermitteln (einen Ordner zurück)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Das übergeordnete Verzeichnis zum Suchpfad hinzufügen
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

print(f"Das Verzeichnis '{parent_dir}' wurde zum sys.path hinzugefügt.")
#from pathlib import Path
from kg_manager_new import KGManagerNew
mgr = KGManagerNew(ttl_path="MSRGuard_Anpassung/KGs/Test_filled.ttl", debug=True)
mgr.run()  # oder einzeln: mgr.load(); mgr.add_calls(); mgr.add_reports(); mgr.save()

Graph geladen mit 778 Tripeln.
[GVL 1/8] http://www.semanticweb.org/AgentProgramParams/Program_FB_Automatikbetrieb_F1
[GVL 2/8] http://www.semanticweb.org/AgentProgramParams/Program_FB_Betriebsarten
Skip http://www.semanticweb.org/AgentProgramParams/Program_FB_Betriebsarten: lang=FBD, code=True
[GVL 3/8] http://www.semanticweb.org/AgentProgramParams/Program_FB_Diagnose_D2
[GVL 4/8] http://www.semanticweb.org/AgentProgramParams/Program_FB_InitFahrt_A6_A2
[GVL 5/8] http://www.semanticweb.org/AgentProgramParams/Program_FB_Methode1Job
[GVL 6/8] http://www.semanticweb.org/AgentProgramParams/Program_FB_Notaus_D1
[GVL 7/8] http://www.semanticweb.org/AgentProgramParams/Program_FB_ProduktionMitStoerung_D3
[GVL 8/8] http://www.semanticweb.org/AgentProgramParams/Program_MAIN
[type-map] http://www.semanticweb.org/AgentProgramParams/Var_MAIN_fbAuto -> Program http://www.semanticweb.org/AgentProgramParams/Program_FB_Automatikbetrieb_F1 (type=FB_Automatikbetrieb_F1)
[type-map] http://www.semanticweb.

WindowsPath('D:/MA_Python_Agent/MSRGuard_Anpassung/KGs/Test_filled.ttl')